[Исходная статья: Causal Discovery with Continuous Additive Noise Models](https://arxiv.org/abs/1309.6779) — *Отличается от первоначально выбранной!*

# Causality

Популярным средством измерения силы причинно-следственной связи процессов, протекающих в природе или обществе, является корреляция. Однако существует принцип, который формулируется довольно просто: "correlation does not mean causation". Пренебрежение им может порождать так называемые «ложные корреляции»: например, количество гнёзд аистов влияет на рождаемость в европейских странах или количество фильмов, в которых снялся Николас Кейдж, может быть каким-то образом связано с гибелью людей в бассейнах.
![Storks](img/storks.png)
![Nicolas Cage](img/cage.png)

Возникает естественный вопрос: в каких случаях "correlation DOES mean causation"? А может, вместо корреляции стоит использовать что-нибудь другое? На этот вопрос пытается ответить [группа исследователей из Института интеллектуальных система Макса Планка.](https://ei.is.tuebingen.mpg.de/research_groups/causal-inference-group) Оказывается, при выполнении некоторых условий, можно восстановить причинно-следственную связь из результатов наблюдений (observational distribution).

Для исследования причинно-следственных связей между процессами [Джуда Пёрл](https://ru.wikipedia.org/wiki/%D0%9F%D0%B5%D1%80%D0%BB,_%D0%94%D0%B6%D1%83%D0%B4%D0%B0) предложил использовать фунцкиональные причинные модели (functional causal models). По уравнениям, описывающим причинную модель, можно построить граф, который отображает зависимости между случайными величинами, но не описывает характера таких зависимостей. Таким образом, граф содержит меньше информации о модели, однако позволяет ввести в терминах теории графов некоторые критерии, упрощающие определение условной независимости между переменными. Эти понятия неразрывно связаны с понятием байесовских сетей.

### Определения

Будем рассматривать конечное семейство случайных величин $X = (X_1, \dots, X_p)$ и соответствующее ему множество индексов $V = \{ 1, \dots, p \}$. Обозначим за $\mathcal{L}(X)$ совместное распределение этих случайных величин.

Пусть граф $\mathcal{G} = (V, \mathcal{E})$ состоит из вершин $V$ и рёбер $\mathcal{E} \subseteq V^2$ и при этом $(v, v) \notin \mathcal{E} ~ \forall v \in V$. Иногда в записи под индексами $j \in V$ подразумеваются соответствующие случайные величины $X_j$.

Вершина $i$ называется вершиной-родителем вершины $j$, если $(i, j) \in \mathcal{E}$, и вершиной-ребёнком, если $(j, i) \in \mathcal{E}$. Множество вершин-родителей вершины $j$ обозначатся $PA_j$, множество вершин-детей — $CH_j$.

Две вершины $i$ и $j$ называются смежными, если $(i, j) \in \mathcal{E}$ или $(j, i) \in \mathcal{E}$. Две вершины $i$ и $j$ соединены ненаправленным ребром, если $(i, j) \in \mathcal{E}$ и $(j, i) \in \mathcal{E}$. Ребро между смежными вершинами называется направленным, если оно не является ненаправленным.

Три вершины называются v-структурой, если одна вершина является ребёнком двух вершин, которые между собой не смежны. Остовом графа $\mathcal{G}$ будем называть набор всех рёбер независимо от их направленности.

Путём в графе $\mathcal{G}$ называется последовательность различных вершин $i_1, ..., i_n$, для которых существуют рёбра между $i_k$ и $i_{k+1}$ для всех $k = 1, \dots, n - 1$. Если $(i_k, i_{k+1}) \in \mathcal{E}$ и $(i_{k+1}, i_{k}) \notin \mathcal{E}$ для всех $k$, то такой путь называется направленным путём между $i_1$ и $i_n$, а $i_n$ называется потомком $i_1$. Множество всех потомков вершины $i$ обозначается $DE_i^{\mathcal{G}}$, множество вершин не-потомков вершины $i$, не включая саму $i$, — $ND_i^{\mathcal{G}}$. Вершина $i$ не является ни потомком, ни не-потомком самой себя.

![Вершины-родители и вершины-дети](img/neighbours.png)

Ориентированным ациклическим графоф (DAG, directed acyclic graph) называется граф, в котором все рёбра ориентированы и нет ни одной пары $(j, k)$ такой, что существуют пути из $j$ до $k$, из $k$ до $j$ одновременно.

Путь между $i_1$ и $i_n$ называется заблокированным множеством $S$ (не включая ни $i_1$, ни $i_n$), если существует такая вершина $i_k$, что выполняется одно из условий:
1. $i_k \in S$ и
  - $i_{k-1} \rightarrow i_k \rightarrow i_{k+1}$ или
  - $i_{k-1} \leftarrow i_k \leftarrow i_{k+1}$ или
  - $i_{k-1} \leftarrow i_k \rightarrow i_{k+1}$
2. $i_{k-1} \rightarrow i_k \leftarrow i_{k+1}$ и ни $i_k$, ни её потомки не принадлежат $S$.

Говорят, что два непересекающихся множества вершин $A$ и $B$ d-разделены (непересекающимся с ними) множеством $S$, если каждый путь между вершинами $A$ и $B$ заблокирован множеством $S$.

![Пример заблокированного пути 1](img/block1.png)

**Пример 1** *Путь из $X$ в $Y$ заблокирован множеством $\{U\}$, множеством $\{Z\}$ или множеством $\{U, Z\}$.*

![Пример заблокированного пути 2](img/block2.png)

**Пример 2** *Путь из $X$ в $Y$ заблокирован пустым множеством.*

Совместное распределение $\mathcal{L}(X)$ называется марковским относительно DAG $\mathcal{G}$, если $A$ и $B$ d-разделены $C \Rightarrow A \!\perp\!\!\!\perp B \mid C$ для всех непересекающихся множеств $A, B, C$. ($\!\perp\!\!\!\perp$ означает условную независимость.) Будем говорить, что распределение удовлетворяет условию причинной минимальности относительно графа $\mathcal{G}$, если оно является марковским относительно $\mathcal{G}$, но не является таковым относительно любого из подграфов $\mathcal{G}$. Обозначим за $\mathcal{M}(\mathcal{G})$ множество распределений, марковских относительно $\mathcal{G}$: $\mathcal{M}(\mathcal{G}) = \{ \mathcal{L} : \mathcal{L} \text{ марковское относительно } \mathcal{G} \}.$ Два графа $\mathcal{G}_1$ и $\mathcal{G}_2$ называются марковски эквивалентными, если $\mathcal{M}(\mathcal{G}_1) = \mathcal{M}(\mathcal{G}_2)$. В этом и только в этом случае $\mathcal{G}_1$ и $\mathcal{G}_2$ удовлетворяют одному и тому же набору d-разделимостей, т. е. имеют одинаковые наборы условных независимостей совместного распределения.

**Лемма.** Два DAG являются марковски эквивалентными тогда и только тогда, когда они имеют одинаковые остовы и одинаковые v-структуры.

Условие причинной минимальности может быть сформулировано с помощью следующего предложения.

**Предложение.** Пусть заданы случайный вектор $X = (X_1, \dots, X_p)$ и совместное распределение на нём $\mathcal{L}$ — марковское относительно $\mathcal{G}$. Тогда $\mathcal{L}$ отвечает причинной минимальности относительно $\mathcal{G}$ тогда и только тогда, когда $\forall X_j ~ \forall Y \in PA_{j}^{\mathcal{G}}$ выполняется $X_j \not\!\perp\!\!\!\perp Y \mid PA_{j}^{\mathcal{G}}\setminus\{Y\}$.

Для всех рассматриваемых графов предполагается ацикличность (отсутствие циклов в причинно-следственных связях) и достаточность (в рассмотрение принимаются все переменные, являющиеся общей причиной двух и более других переменных).

Следующее определение вводится для того, чтобы формализовать понятие «истинного графа», на самом деле лежащего в основе наблюдаемого процесса. Оно впервые предложено авторами статьи как попытка строго определить понятие, которым пользуются многие, заметая формальности под ковёр. Для строгого определения потребуется так называемая do-нотация, предложенная Джудой Пёрлом.

**Определение.** Пусть $X = (X_1, \dots, X_p)$ — семейство случайных величин с совместным распределением $\mathcal{L}$, абсолютно непрерывным относительно меры Лебега. Для направленного ациклического графа $\mathcal{G}$, заданного над $X$, будем называть интервенционным распределением (interventional distribution) $do(X_j =\tilde{p}(x_j))$ величин $X_1, \dots, X_p$
$$p(x_1, \dots, x_p \mid do(X_j =\tilde{p}(x_j))) = \prod_{i \ne j}^p p(x_i \mid x_{PA_i} ) \cdot\tilde{p}(x_j),$$
если $p(x_1, \dots, x_p) > 0$, и 0 в другом случае.

Иначе говоря, запись $do(X_j =\tilde{p}(x_j ))$ означает назначение значения переменной $X_j$ случайным образом из распределения $\tilde{p}(x_j)$ независимо от значения её родителей.

Под $x_{PA_i}$ понимается набор всех $X_j$, которые являются непосредственными родителями $X_i$ в графе $\mathcal{G}$.

Выражение $p(x_1, \dots, x_p \mid do(X_j =\tilde{x}_j, j \in J))$ задаёт распределение над $X_1, \dots, X_p$.

**Определение.** Пусть даны $\mathcal{L}(X)$ — распределение над $X_1, \dots, X_p$ — и распределения $\mathcal{L}_{do(X_j=\tilde{p}(x_j),j \in J)}(X)$ для всех $J \subset V = {1, \dots, p}$. Будем называеть граф $\mathcal{G}_c$ истинным причинным графом, если:
- $\mathcal{G}_c$ — это направленный ациклический граф,
- распределение $\mathcal{L}(X)$ — марковское относительно $\mathcal{G}_c$;
- для всех $J \subseteq V$ и $\tilde{p}(x_j), j \in J$ распределение $\mathcal{L}_{do(X_j = \tilde{p}(x_j), j \in J)}(X)$ совпадает с $p(x_1, \dots, x_p \mid do(X_j = \tilde{p}(x_j)), j \in J),$ вычисленным для $\mathcal{G}_c$ в предыдущем определении.

Вообще говоря, может существовать несколько истинных причинных графов. Если наложить условие причинной минимальности, истинный граф определяется единственным образом.

### Функциональные модели

**Определение.** Structural equation model (SEM), или *функциональной моделью* называется пара $(\mathcal{S},\mathcal{L}(N))$, где $\mathcal{S}=(S_1, \dots,S_p)$ — набор из $p$ уравнений

$$S_j: \quad X_j = f_i(PA_j, N_j), \quad j = 1, \dots, p$$

и $\mathcal{L}(N)=\mathcal{L}(N_1,\dots,N_p)$ — совместное распределение шумовых переменных, от которых требуется независимость в совокупности.

По функциональной модели можно построить граф, в котором каждая вершина имеет родителями лишь те вершины, которые оказывают непосредственное влияние на неё (стоят в правой части соответствующего уравнения в функциональной модели). Очевидно, что граф не отображает формы этой зависимости, он лишь свидетельствует о наличии прямой связи. Если в уравнениях нет связи между двумя случайными величинами, то граф не содержит ребра между соответствующими вершинами. Очевидно, что такой граф содержит меньше информации, чем функциональная модель.

**Пример.** Пусть SEM (функциональная модель) задана в виде

$$X_1 = 1,$$
$$X_2 = 4,$$
$$X_3 = X_1 + 2 X_2,$$
$$X_4 = 3 X_3,$$
$$X_5 = 7 X_4,$$
$$X_6 = X_4 + 4.$$

Соответствующий ей граф:

![Граф, соответствующий функциональной модели](img/graph.png)

do-нотация подразумевает, что все выражения, в которых участвует случайная величина $X$ под оператором do$(X = x)$, принимает указанное значение $x$ во всех уравнениях, в которых она появляется, и таким образом оказывает влияние на значения переменных, от неё зависящих. Например, do$(X_3 = 1)$ означает, что функциональная модель принимает следующий вид:

$$X_1 = 1,$$
$$X_2 = 4,$$
$$X_3 = 1,$$
$$X_4 = 3 \times 1,$$
$$X_5 = 7 X_4,$$
$$X_6 = X_4 + 4.$$

<sub>Вообще do-нотация изначально предложена Джудой Пёрлом для ответа на [контрафактуальные](https://en.wikipedia.org/wiki/Counterfactual_conditional) вопросы. Например, в эксперименте, изучающем влияние курения на организм человека, запись do(X = курить) означает, что все участники эксперимента, включая женщин и детей, начинают курить, т. е. подразумевает непосредственное вмешательство в эксперимент. Вопросы, связанные с ответом на сложные вопросы без проведения прямых экспериментов (— Мы подали большое напряжение, и лампочка перегорела. А перегорела ли бы она, если бы мы подали чуть меньшее напряжение?), относятся к теме Causal inference и в данном конспекте не рассматриваются.</sub>

Функциональная модель задаёт, каким образом $PA_j$ оказывают влияние на $X_j$.

Граф, полученный из функциональной модели, позволяет факторизовать совместную плотность распределения по следующей формуле:

$$\mathcal{L}(X)=\mathcal{L}(X_1, \dots, X_p) = \prod_j p(X_j \mid pa_j).$$

Т. е., достаточно рассматривать непосредственных родителей каждой из вершин.

![Пример факторизации распределения графа](img/factorize.png)

Пример факторизации распределения: $p(X, Y, Z) = p(Z) p(X|Z) P(Y|X, Z)$

**Предложение.** Пусть $X_1, \dots, X_p$ — случайные величины, $\mathcal{L}(X)$ — их совместное распределение относительно $\mathcal{G}$. Тогда существуют SEM $(\mathcal{S},\mathcal{L}(N))$ и граф $\mathcal{G}$, порождающие распределение $\mathcal{L}(X)$.

### Математическая постановка задачи.

Дана выборка из совместного распределения $\mathcal{L} = \mathcal{L}(X_1, \dots, X_p)$, порождённого (неизвестной) функциональной модели с графом $\mathcal{G}_0$, при этом $\mathcal{L}(X)$ — марковское относительно $\mathcal{G}_0$.

Требуется восстановить граф $\mathcal{G}_0.$ *(Иногда такая задача называется structure learning.)*

### Идентифицируемость непрерывных моделей с аддитивным шумом

#### Идентифицируемость

Дополнительно вводится определение идентифицируемости.

**Определение.** Пусть распределение $\mathcal{L}(X)=\mathcal{L}(X_1, \dots, X_p)$ получено по (неизвестной) функциональной модели с графом $\mathcal{G}_0$, в частности, $\mathcal{L}(X)$ марковское относительно $\mathcal{G}_0$. Будем называеть $\mathcal{G}_0$ идентифицируемым по $\mathcal{L}(X)$, если распределение $\mathcal{L}(X)$ не могло быть получено по той же функциональной модели с графом $\mathcal{G} \ne \mathcal{G}_0$.

Как правило, $\mathcal{G}_0$ не идентифицируем по $\mathcal{L}(X)$, поскольку совместное распределение $\mathcal{L}(X)$ обычно является марковским относительно большого количества различных графов. Однако наложение некоторых дополнительных требований позволяет добиться идентифицируемости ограниченных моделей.

Рассмотрим функциональные модели следующего вида: $X_j = f_j(PA_j) + N_j, j = 1, \dots, p$, где $N_j$ имеют строго положительные распределения.

Для таких моделей требование причинной минимальности упрощается.

**Предложение.** Пусть в описанной модели функции $f_i$ не являются постоянными ни по одному из их аргументов. Тогда совместное распределение, полученное по этой модели, отвечает требованию причинной минимальности относительно соответствующего графа.

#### Модели с аддитивным шумом двух переменных

Идентифицируемость моделей с аддитивным шумом двух переменных возможна при выполнении любопытного условия.

**Условие 1.** Модель двух переменных с аддитивным шумом $X_1 = N_1$ и $X_2 = f_2(X_1) + N_2$ является идентифицируемой, если тройка $(f_2, \mathcal{L}(X_1), \mathcal{L}(N_2))$ не является решением дифференциального уравнения
$$\xi''' = \xi'' \left( -\frac{\nu''' f'}{\nu''} + \frac{f''}{f'} \right) - 2 \nu'' f'' f' + \nu' f''' + \frac{\nu' \nu''' f'' f'}{nu''} - \frac{nu' (f'')^2}{f'},$$
где $f = f_2, \xi = \log{p_{X_1}}, \nu = \log{p_{N_2}}.$

Идентифируемость графа, лежащего в основе исследуемой модели, гарантируется следующей теоремой.

**Теорема.** Пусть $\mathcal{L}(X) = \mathcal{L}(X_1, X_2)$ порождена идентифицируемой моделью двух переменных с аддитивным шумом с графом $\mathcal{G}_0$ и выполняется условие причинной минимальности. Тогда $\mathcal{G}_0$ идентифицируем по совместному распределению.

Результаты, полученные для модели двух переменных, могут быть обобщены для случая нескольких переменных.

#### Модели с аддитивным шумом нескольких переменных

Пусть теперь заданы $p$ уравнений вида $X_j = f_j(PA_j) + N_j.$

**Определение.** Будем называеть SEM ограниченной моделью с аддитивным шумом, если для всех $j \in V, i \in PA_j$ и множеств $S \subseteq V$, для которых $PA_j \ { i } \subseteq S \subseteq ND_j \ {i, j}$, существует $x_S: p_S(x_S) > 0$ такой, что $$\left( f_j(x_{PA_{j} \ {i}}, \cdot), \mathcal{L}(X_i \mid X_S = x_S), \mathcal{L}(N_j) \right)$$ удовлетворяет **условию 1.**

**Теорема.** Пусть $\mathcal{L}(X)=\mathcal{L}(X_1, \dots, X_p)$ порождено ограниченой моделью с аддитивным шумом и графом $\mathcal{G}_0$ и выполняет условие причинной минимальности относительно $\mathcal{G}_0$ (т. е. функции $f_j$ не постоянны). Тогда $\mathcal{G}_0$ идентифицируем по совместному распределению.

### Алгоритм RESIT (Regression with Subsequent Independence Test)

На практике имеется конечный набор i.i.d. данных, полученный из совместного распределения, по которому требуется оценить граф, порождающий это распределение.

Предлагается следующая итерационная процедура: на каждом шаге обнаруживается и удаляется из рассмотрения вершина-сток (sink, вершина, из которой нет исходящих рёбер). Это осуществляется путём регрессии каждой из оставшихся вершин (переменных) на все другие переменных и оценки меры зависимости между остатками и переменными. Переменная, для которой получена минимальная оценка зависимости остатков, считается стоком.

Первый этап алгоритма определяет причинное упорядочивание вершин полного графа, на втором этапе у каждой вершины удаляются лишние входящие рёбра до тех пор, пока остатки регрессии не станут независимыми.

Алгоритм предоставляет выбор способов построения модели регрессии и тестирования независимости отстатков. Авторы рекомендуют использовать модели линейной и нелинейной регрессии (GAM), а также Gaussian
process regression, а для меры зависимости — p-значение теста независимости [Hilbert-Schmidt independence criterion](https://papers.nips.cc/paper/3201-a-kernel-statistical-test-of-independence.pdf).

![Algo](img/algo.png)

**Теорема.** Пусть $\mathcal{L}(X) = \mathcal{L}(X_1, \dots, X_p)$ порождено ограниченной моделью с аддитивным шумом и графом $\mathcal{G}_0$ и выполняется условие причинной минимальности относительно графа $\mathcal{G}_0$. Тогда алгоритм RESIT, использующий соответствующую модель регрессии и «оракул» независимости (метод, с абсолютной точностью определяющий независимость входных аргументов) гарантированно находит правильный граф $\mathcal{G}_0$ по совместному распределению $\mathcal{L}(X)$.

Алгоритм выполняет порядка $\mathcal{O}(p^2)$ тестов независимости (полиномиальная сложность в зависимости от количества вершин); удаление лишних рёбер выполняется за $\mathcal{O}(p)$. Процедура становится вычислительно сложной для графов с большим количеством переменных. Помимо прочего, на практике в нашем распоряжении нет оракула, определяющего независимость, поэтому её придётся определять из конечных наборов данных. Для приемлемой точности определение независимости в задачах большой размерности потребует быстро растущих объёмов выборки.

### Существующие алгоритмы

*PC-алгоритм* (Peter-Clark) находит частично ориентированное остовное дерево причинного графа, соответствующего классу марковской эквивалентности истинного графа. Алгоритм основан на тестировании условной независимости (CIB, conditional independence-based), его описание можно найти в главе 25 книги "Causation, Prediction, and Search" Peter Spirtes, Clark Glymour, and Richard Scheines (профессоров философии (sic!) университета Карнеги — Меллон). Имеется реализация алгоритма на языке R в пакете `pcalg` (функция `pc`). Такие методы различают причинные графы до класса марковской эквивалентности, поэтому некоторые рёбра могут остаться ненаправленными.

*Score-Based* методы вводят специальную оценку соответствия графа распределению $S(\mathcal{D}, \mathcal{G}),$ а затем некоторым образом максимизируют её:  $$\underset{\mathcal{G} \text{ DAG на } X }{\arg\!\max} S(\mathcal{D}, \mathcal{G}).$$

Метод *LiNGAM* использует ограничения на распределение шумовых переменных.

Теорема (Peters and Bühlmann, 2014). Пусть SEM с графом $\mathcal{G}_0$ состоит из
$$X_j = \sum_{k \in PA_{j}^{\mathcal{G}_0}} \beta_{jk} X_k + N_j, ~ j = 1, \dots, p,$$
где все $N_j$ независимы в совокупности и распределены не нормально, и для $j \in \{ 1, \dots, p \} \quad \beta_{jk} \ne 0$ для всех $k \in PA_{j}^{\mathcal{G}_0}$. Тогда граф $\mathcal{G}_0$ идентифицируем по совместному распределению.

### Тестирование на модельных данных

Авторы проверяют точность работы алгоритма на тестовых наборах данных, сгенерированных для линейных и нелинейных $f_i$ в SEM. Эксперименты показывают, что RESIT оказывается лучше метода PC, а в случае $p = 15$ переменных справляется с определением структуры графа для нелинейных SEM.

Упрощённая версию алгоритма была протестирована на [коллекции пар «причина—следствие»](http://webdav.tuebingen.mpg.de/cause-effect/), полученных из различных предметных областей. Для каждой пары требовалось указать, какая переменная является непосредственной причиной другой. Предложенный алгоритм показал точность около 80% на наборе из 86 пар. (Похожим образом была поставлена задача в одном из соревнований Kaggle: https://www.kaggle.com/c/cause-effect-pairs/.)

### Возможные проблемы

Теоретический результат подразумевает существование “оракула”, который идеально определяет независимость или зависимость остатков. На практике такого оракула в распоряжении нет, поэтому нужен хороший тест независимости (возможно, придётся воспользоваться некоторой информацией о специфике наблюдаемых процессов) и большой объём данных. Последнее, в свою очередь, ведёт к столкновению с проклятием размерности.

Полученные резульаты накладывают серьёзные ограничения на анализируемую модель (линейные зависимости, аддитивный шум с положительной плотностью распределения). На практике трудно найти такой процесс, который в самом деле подчинялся этим условиям.

Выбор алгоритма регрессии и теста независимости целиком возлагается на исследователя, применяющего алгоритм.

Оценок по количеству нужных данных для хорошего качества работы алгоритма авторы не приводят, их нужно получать после численного эксперимента для каждого конкретного алгоритма регрессии с каждым конкретным тестом независимости.

## Источники

1. [Jonas Peters, Joris Mooij, Dominik Janzing, Bernhard Schölkopf "Causal Discovery with Continuous Additive Noise Models"](https://arxiv.org/abs/1309.6779)
2. [Dominik Janzing, Bernhard Schölkopf "Causal Inference Conditional Independences and Beyond"](http://people.kyb.tuebingen.mpg.de/janzing/ICML_Tutorial/JanzingSchoelkopf.pdf)
3. [Peter Spirtes, Clark Glymour, and Richard Scheines "Causation, Prediction, and Search"](https://www.cs.cmu.edu/afs/cs.cmu.edu/project/learn-43/lib/photoz/.g/scottd/fullbook.pdf)
4. [Arthur Gretton, Kenji Fukumizu и др. "A Kernel Statistical Test of Independence"](https://papers.nips.cc/paper/3201-a-kernel-statistical-test-of-independence.pdf)
5. Judea Pearl "Causality: Models, Reasoning, and Inference"